In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics 
import sklearn.preprocessing as preprocessing

In [2]:
df_train = pd.read_csv('tmlcc-2021/train.csv')
df_test = pd.read_csv('tmlcc-2021/test.csv')
df_pre_train = pd.read_csv('tmlcc-2021/pretest.csv')

In [3]:
df = df_train.iloc[:, [1, 2, 3, 4, 5, 11, 12, 13]].sample(n=3000)

In [4]:
df.isnull().sum(axis=0)

volume [A^3]                                       0
weight [u]                                         0
surface_area [m^2/g]                               0
void_fraction                                      0
void_volume [cm^3/g]                               0
CO2/N2_selectivity                                 0
heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]    110
CO2_working_capacity [mL/g]                        0
dtype: int64

In [5]:
Max_heat = df['heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]'].replace([np.inf], 0).max()

In [6]:
df['heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]'].replace([np.inf], Max_heat, inplace=True)

In [7]:
df['heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]'].fillna(df["heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]"].median(), inplace=True)

In [8]:
target = df.pop("CO2_working_capacity [mL/g]")
data = df   

In [9]:
# scale = preprocessing.MinMaxScaler()
# scale.fit(data)
# data = scale.transform(data)
# data = pd.DataFrame(data, columns=['a','b','c','d', 'e','f','g'])

In [10]:
data.columns = ['a','b','c','d', 'e','f','g']
data.head()

,a,b,c,d,e,f,g
55947,1770.283712,1748.36860,0.00,0.11372,0.0693,37.059345,6.889322
64126,14405.415234,5512.48912,2828.07,0.50969,0.8021,5.533757,3.432523
54759,1099.849047,741.37836,0.00,0.09711,0.0868,35.539486,6.352950
28624,1118.021129,740.27125,1037.19,0.17276,0.1571,28.046036,6.182082
32736,1641.623774,1187.45612,0.00,0.17162,0.1429,38.229134,6.410619


In [11]:
target

55947    156.504783
64126      5.996075
54759    128.785129
28624    121.511031
32736     96.593221
            ...    
35992    305.350345
51367    208.279392
58717     87.791505
56343     73.868619
25906     56.816211
Name: CO2_working_capacity [mL/g], Length: 3000, dtype: float64

### Operator

In [12]:
import operator                                                                                      
                                                                                                     
val1 = {"feature_name": "a"}                                                                
val2 = {"feature_name": "b"}                                                                 
val3 = {"feature_name": "c"}   
val4 = {"feature_name": "d"}  
val5 = {"feature_name": "e"}  
val6 = {"feature_name": "f"} 
val7 = {"feature_name": "g"}

node1 = {                                                                                            
    "func": operator.add,                                                                            
    "children": [val1, val2],                                                                        
    "format_str": "({} + {})",                                                                       
}                                                                                                    
program = {                                                                                          
    "func": operator.mul,                                                                            
    "children": [node1, val3],                                                                       
    "format_str": "({} * {})",                                                                       
}

In [13]:
def render_prog(node):                                                                               
    if "children" not in node:                                                                       
        return node["feature_name"]                                                                  
    return node["format_str"].format(*[render_prog(c) for c in node["children"]])  

In [14]:
print(render_prog(program))

((a + b) * c)


In [15]:
def safe_div(a, b):                                                                                  
    return a / b if b else a                                                                         

operations = (                                                                                       
    {"func": operator.add, "arg_count": 2, "format_str": "({} + {})"},                               
    {"func": operator.sub, "arg_count": 2, "format_str": "({} - {})"},                               
    {"func": operator.mul, "arg_count": 2, "format_str": "({} * {})"},                               
    {"func": safe_div, "arg_count": 2, "format_str": "({} / {})"},                                   
    {"func": operator.neg, "arg_count": 1, "format_str": "-({})"},    
)

In [16]:
def evaluate(node, row):                                                                             
    if "children" not in node:                                                                       
        return row[node["feature_name"]]                                                             
    return node["func"](*[evaluate(c, row) for c in node["children"]])  

In [17]:
print(evaluate(program, data.iloc[0]))

0.0


### GP

In [18]:
from random import randint, random, seed                                                             
                                                                                                     
seed(0)                                                                                              
                                                                                                     
def random_prog(depth):                                                                              
    # favor adding function nodes near the tree root and 
    # leaf nodes as depth increases                           
    if randint(0, 10) >= depth * 2:                                                                  
        op = operations[randint(0, len(operations) - 1)]                                             
        return {                                                                                     
            "func": op["func"],                                                                      
            "children": [random_prog(depth + 1) for _ in range(op["arg_count"])],                    
            "format_str": op["format_str"],                                                          
        }                                                                                            
    else:                                                                                            
        return {"feature_name": data.columns[randint(0, data.shape[1] - 1)]}                         
                                                                                                     
                                                                                                     
POP_SIZE = 300                                                                                      
population = [random_prog(0) for _ in range(POP_SIZE)] 

In [19]:
print(render_prog(population[0]))   

(c / ((((b - b) * e) * e) / (a - g)))


In [20]:
def select_random_node(selected, parent, depth):                                                     
    if "children" not in selected:                                                                   
        return parent                                                                                
    # favor nodes near the root                                                                      
    if randint(0, 10) < 2*depth:                                                                     
        return selected                                                                              
    child_count = len(selected["children"])                                                          
    return select_random_node(
        selected["children"][randint(0, child_count - 1)], 
        selected, depth+1) 

In [21]:
print(render_prog(select_random_node(program, None, 0)))

((a + b) * c)


In [22]:
from copy import deepcopy
 
def do_mutate(selected):
    offspring = deepcopy(selected)
    mutate_point = select_random_node(offspring, None, 0)
    child_count = len(mutate_point["children"])
    mutate_point["children"][randint(0, child_count - 1)] = random_prog(0)
    return offspring

In [23]:
print(render_prog(do_mutate(program)))

(((-(b) - (d / (g / d))) + b) * c)


In [24]:
def do_xover(selected1, selected2):                                                                  
    offspring = deepcopy(selected1)                                                                  
    xover_point1 = select_random_node(offspring, None, 0)                                            
    xover_point2 = select_random_node(selected2, None, 0)                                            
    child_count = len(xover_point1["children"])                                                      
    xover_point1["children"][randint(0, child_count - 1)] = xover_point2                             
    return offspring       

In [25]:
print(render_prog(do_xover(population[0], population[1])))

(c / (((-(c) * e) * e) / (a - g)))


In [26]:
TOURNAMENT_SIZE = 3                                                                                  
                        
def get_random_parent(population, fitness):                                                          
    # randomly select population members for the tournament                                          
    tournament_members = [
        randint(0, POP_SIZE - 1) for _ in range(TOURNAMENT_SIZE)]                  
    # select tournament member with best fitness                                                     
    member_fitness = [(fitness[i], population[i]) for i in tournament_members]                       
    return min(member_fitness, key=lambda x: x[0])[1]

In [27]:
XOVER_PCT = 0.7                                                                                                                                                            
                                                                                                     
def get_offspring(population, fitness):                                                              
    parent1 = get_random_parent(population, fitness)                                                 
    if random() > XOVER_PCT:                                                                         
        parent2 = get_random_parent(population, fitness)                                             
        return do_xover(parent1, parent2)                                                            
    else:                                                                                            
        return do_mutate(parent1) 

In [28]:
REG_STRENGTH = 0.5                                                                                   

def node_count(x):                                                                                   
    if "children" not in x:                                                                          
        return 1                                                                                     
    return sum([node_count(c) for c in x["children"]])  

def compute_fitness(program, prediction):                                                            
    #mse = ((pd.Series(prediction) - target) ** 2).mean()                                             
    #penalty = node_count(program) ** REG_STRENGTH                                                    
    #return mse * penalty 
    return np.log(metrics.mean_absolute_error(target, prediction))

In [29]:
MAX_GENERATIONS = 5                                                                                 
                                                                                                     
global_best = float("inf")                                                                           
for gen in range(MAX_GENERATIONS):                                                                   
    fitness = []                                                                                     
    for prog in population:                                                                          
        prediction = [                                                                               
            evaluate(prog, row) for _, row in data.iterrows()]                                       
        score = compute_fitness(prog, prediction)                                                    
        fitness.append(score)                                                                        
        if score < global_best:                                                                      
            global_best = score                                                                      
            best_pred = prediction                                                                   
            best_prog = prog                                                                         
    print(                                                                                           
        "Generation: %d\nBest Score: %.2f\nMedian score: %.2f\nBest program: %s\n"                   
        % (                                                                                          
            gen,                                                                                     
            global_best,                                                                             
            pd.Series(fitness).median(),                                                             
            render_prog(best_prog),                                                                  
        )                                                                                            
    )                                                                                                
    population = [                                                                                   
        get_offspring(population, fitness)                                                           
        for _ in range(POP_SIZE)]                                                                                                             


Generation: 0
Best Score: 4.10
Median score: 7.74
Best program: -(((-(g) + d) * ((g - d) + g)))

Generation: 1
Best Score: 4.10
Median score: 6.17
Best program: -(((-(g) + d) * ((g - d) + g)))

Generation: 2
Best Score: 4.10
Median score: 4.95
Best program: -(((-(g) + d) * ((g - d) + g)))

Generation: 3
Best Score: 4.10
Median score: 4.79
Best program: -(((-(g) + d) * ((g - d) + g)))

Generation: 4
Best Score: 4.10
Median score: 4.78
Best program: -(((-(g) + d) * ((g - d) + g)))



In [30]:
print("Best score: %f" % global_best)                                                                
print("Best program: %s" % render_prog(best_prog))                                                   
output = {"target": target, "pred": best_pred}                                                       
# pd.DataFrame(output).to_csv("best_pred.csv")  

Best score: 4.097968
Best program: -(((-(g) + d) * ((g - d) + g)))


In [31]:
evaluate(best_prog, data.iloc[4])

78.92095405238202

In [32]:
target

55947    156.504783
64126      5.996075
54759    128.785129
28624    121.511031
32736     96.593221
            ...    
35992    305.350345
51367    208.279392
58717     87.791505
56343     73.868619
25906     56.816211
Name: CO2_working_capacity [mL/g], Length: 3000, dtype: float64